# Container Orchestration with Kubernetes

## Concept 02: Transitions from VMs to Containers

<img src="imgs/le03ex02im01.png" title="The transition from VMs to containers"  />

## Concept 03: Docker for Application Packaging

3 main components are distinguished: **Dockerfiles**, **Docker images**, and **Docker registries**.

### New terms
- **Dockerfile** - set of instructions used to create a Docker image
- **Docker image** - a read-only template used to spin up a runnable instance of an application
- **Docker registry** - a central mechanism to store and distribute Docker images

### Further reading
Explore Dockerfiles best practices and valid list of instructions:
- [Dockerfile reference](https://docs.docker.com/engine/reference/builder/#from)
- [Best practices for writing Dockerfiles](https://docs.docker.com/develop/develop-images/dockerfile_best-practices/)

Explore how to build and run a Docker image, with a list of all available options:
- [Docker Build command](https://docs.docker.com/engine/reference/commandline/build/)
- [Docker Run command](https://docs.docker.com/engine/reference/commandline/run/)

Explore Docker registries, alternatives to package an application, and OCI standards:
- [Introduction to Docker registry](https://docs.docker.com/registry/introduction/)
- [Docker Tag command](https://docs.docker.com/engine/reference/commandline/tag/)
- [Docker Push command](https://docs.docker.com/engine/reference/commandline/push/)
- [Demystifying the Open Container Initiative (OCI) Specifications](https://www.docker.com/blog/demystifying-open-container-initiative-oci-specifications/)
- [Buildpacks: An App’s Brief Journey from Source to Image](https://buildpacks.io/docs/app-journey/)

### Dockerfile

A Dockerfile is a set of instructions used to create a Docker image. Each instruction is an operation used to package the application, such as installing dependencies, compile the code, or impersonate a specific user. A Docker image is composed of multiple layers, and each layer is represented by an instruction in the Dockerfile. **All layers are cached and if an instruction is modified, then during the build process only the changed layer will be rebuild**. As a result, building a Docker image using a Dockerfile is a lightweight and quick process.

```docker
FROM -  to set the base image
RUN - to execute a command
COPY & ADD  - to copy files from host to the container
CMD - to set the default command to execute when the container starts
EXPOSE - to expose an application port 
```

```dockerfile
# set the base image. Since we're running 
# a Python application a Python base image is used
FROM python:3.8
# set a key-value label for the Docker image
LABEL maintainer="Katie Gamanji"
# copy files from the host to the container filesystem. 
# For example, all the files in the current directory
# to the  `/app` directory in the container
COPY . /app
#  defines the working directory within the container
WORKDIR /app
# run commands within the container. 
# For example, invoke a pip command 
# to install dependencies defined in the requirements.txt file. 
RUN pip install -r requirements.txt
# provide a command to run on container start. 
# For example, start the `app.py` application.
CMD [ "python", "app.py" ]
```

### Docker Image

A Docker image is a read-only template that enables the creation of a runnable instance of an application.

A Docker image can be built from an existing Dockerfile using the docker build command. Below is the syntax for this command:
```bash
# build an image
# OPTIONS - optional;  define extra configuration
# PATH - required;  sets the location of the Dockefile and  any referenced files 
docker build [OPTIONS] PATH

# Where OPTIONS can be:
-t, --tag - set the name and tag of the image
-f, --file - set the name of the Dockerfile
--build-arg - set build-time variables

# Find all valid options for this command 
docker build --help
```

For example, to build the image of the Python hello-world application from the Dockerfile, the following command can be used:
```bash
# build an image using the Dockerfile from the current directory
docker build -t python-helloworld .

# build an image using the Dockerfile from the `lesson1/python-app` directory
docker build -t python-helloworld lesson1/python-app
```

To create a container using an available Docker image, the docker run command is available. Below is the syntax for this command:
```bash
# execute an image
# OPTIONS - optional;  define extra configuration
# IMAGE -  required; provides the name of the image to be executed
# COMMAND and ARGS - optional; instruct the container to run specific commands when it starts 
docker run [OPTIONS] IMAGE [COMMAND] [ARG...]

# Where OPTIONS can be:
-d, --detach - run in the background 
-p, --publish - expose container port to host
-it - start an interactive shell

# Find all valid options for this command 
docker run --help
```

For example, to run the Python hello-world application, using the created image, the following command can be used:

**Note**: To access the application in a browser, we need to bind the Docker container port to a port on the host or local machine. In this case, `5111` is the host port that we use to access the application e.g. `http://127.0.0.1:5111/`. The `5000` is the container port that the application is listening to for incoming requests.

```bash
# run the `python-helloworld` image, in detached mode and expose it on port `5111`
docker run -d -p 5111:5000 python-helloworld
```

To retrieve the Docker container logs use the `docker logs {{ CONTAINER_ID }}` command. For example:
```bash
docker logs 95173091eb5e

## Example output from a Flask application
 * Serving Flask app "app" (lazy loading)
 * Environment: production
   WARNING: This is a development server. Do not use it in a production deployment.
   Use a production WSGI server instead.
 * Debug mode: off
```

### Docker Registry

The last step in packaging an application using Docker is to store and distribute it. So far, we have built and tested an image on the local machine, which does not ensure that other engineers have access to it. As a result, the image needs to be pushed to a public Docker image registry, such as DockerHub, Harbor, Google Container Registry, and many more.

Before pushing an image to a Docker registry, it is highly recommended to tag it first. During the build stage, if a tag is not provided (via the `-t` or `--tag` flag), then the image would be allocated an ID, which does not have a human-readable format (e.g. 0e5574283393). On the other side, a defined tag is easily scalable by the human eye, as it is composed of a registry repository, image name, and version. Also, a tag provides version control over application releases, as a new tag would indicate a new release.

To tag an existing image on the local machine, the `docker tag` command is available. Below is the syntax for this command:
```bash
# tag an image
# SOURCE_IMAGE[:TAG]  - required and the tag is optional; define the name of an image on the current machine 
# TARGET_IMAGE[:TAG] -  required and the tag is optional; define the repository, name, and version of an image
docker tag SOURCE_IMAGE[:TAG] TARGET_IMAGE[:TAG]
```

For example:
```bash
# tag the `python-helloworld` image, to be pushed 
# in the `pixelpotato` repository, with the `python-helloworld` image name
# and version `v1.0.0`
docker tag python-helloworld pixelpotato/python-helloworld:v1.0.0
```

Once the image is tagged, the final step is to push the image to a registry. For this purpose, the `docker push` command can be used. Below is the syntax for this command:
```bash
# push an image to a registry 
# NAME[:TAG] - required and the tag is optional; name, set the image name to be pushed to the registry
docker push NAME[:TAG]
```

For example:
```bash
# push the `python-helloworld` application in version v1.0.0 
# to the `pixelpotato` repository in DockerHub
docker push pixelpotato/python-helloworld:v1.0.0
```

## Concept 05: Useful Docker Commands

**Note**: In the following commands the following arguments are used:
- `OPTIONS` - define extra configuration through flags
- `IMAGE` - sets the name of the image
- `NAME` - set the name of the image
- `COMMAND` and `ARG` - instruct the container to run specific commands associated with a set of arguments

```bash
# Build image
docker build [OPTIONS] PATH

# Run image
docker run [OPTIONS] IMAGE [COMMAND] [ARG...]

# Get logs
docker logs CONTAINER_ID

# List images
docker images

# List containers
docker ps

# Tag Images
docker tag SOURCE_IMAGE[:TAG] TARGET_IMAGE[:TAG]

# Login to DockerHub
docker login

# Push image
docker push NAME[:TAG]

# Pull image
docker pull NAME[:TAG]
```

## Concept 09: Kubernetes - The Container Orchestrator Framework

### New terms
- **CRD** - Custom Resource Definition provides the ability to extend Kubernetes API and create new resources
- **Node** - a physical or virtual server
- **Cluster** - a collection of distributed nodes that are used to manage and host workloads
- **Master node** - a node from the Kubernetes control plane, that has installed components to make global, cluster-level decisions
- **Worker node** - a node from the Kubernetes data plane, that has installed components to host workloads

### Further Reading
Explore Kubernetes features:
- [Kubernetes DNS for Services and Pods](https://kubernetes.io/docs/concepts/services-networking/dns-pod-service/)
- [Kubernetes CRDs](https://kubernetes.io/docs/concepts/extend-kubernetes/api-extension/custom-resources/)
- [Kubernete Cluster Autoscaler](https://kubernetes.io/blog/2016/07/autoscaling-in-kubernetes/)
- [Kubernetes Architecture and Components](https://kubernetes.io/docs/concepts/overview/components/)
---

Kubernetes analogs:
- Docker Swarm
- Apache Mesos
- CoreOS Fleet

#### Portability
Kubernetes is a highly portable tool. This is due to its open-source nature and vendor agnosticism. As such, Kubernetes can be hosted on any available infrastructure, including public, private, and hybrid cloud.

#### Scalability
Building for scale is a cornerstone of any modern infrastructure, enabling an application to scale based on the amount of incoming traffic. Kubernetes has in-build resources, such as HPA (Horizontal Pod Autoscaler), to determine the required amount of replicas for a service. Elasticity is a core feature that is highly automated within Kubernetes.

#### Resilience
Failure is expected on any platform. However, it is more important to be able to recover from failure fast and build a set of playbooks that minimizes the downtime of an application. Kubernetes uses functionalities like ReplicaSet, readiness, and liveness probes to handle most of the container failures, which enables powerful self-healing capability.

#### Service Discovery
Service discovery encapsulates the ability to automatically identify and reach new services once these are available. Kubernetes provide cluster level DNS (or Domain Name System), which simplifies the accessibility of workloads within the cluster. Additionally, Kubernetes provides routing and load balancing of incoming traffic, ensuring that all requests are served without application overload.

#### Extensibility
Kubernetes is a highly extensible mechanism that uses the building-block principle. It has a set of basic resources that can be easily adjusted. Additionally, it provides a rich API interface, that can be extended to accommodate new resources or CRDs (Custom Resource Definitions).

#### Operational Cost
Operational cost refers to the efficiency of resource consumption within a Kubernetes cluster, such as CPU and memory. Kubernetes has a powerful scheduling mechanism that places an application on the node with sufficient resources to ensure the successful execution of the service. As a result, most of the available infrastructure resources are allocated on-demand. Additionally, it is possible to automatically scale the size of the cluster based on the current incoming traffic. This capability is provisioned by the cluster-autoscaler, which guarantees that the cluster size is directly proportional to the traffic that it needs to handle.

---

### Kubernetes Architecture

<img src="imgs/le03ex09im01.png" title="Kubernetes architecture, composed of control and data planes"  />

A Kubernetes cluster is composed of a collection of distributed physical or virtual servers. These are called **nodes**. Nodes are categorized into 2 main types: 
- **Master**: the suite of master nodes, represents the **control plane**;
- **Worker nodes**: the collection of worker nodes constructs the **data plane**.

The components installed on a node, determine the functionality of a node, and identifies it as a master or worker node.

### Control Plane

<img src="imgs/le03ex09im02.png" title="Control Plane components"  />

The control plane consists of components that make global decisions about the cluster. These components are the:
- **kube-apiserver**: the nucleus of the cluster that exposes the Kubernetes API, and handles and triggers any operations within the cluster
- **kube-scheduler**: the mechanism that places the new workloads on a node with sufficient satisfactory resource requirements
- **kube-controller-manager**: the component that handles controller processes. It ensures that the desired configuration is propagated to resources
- **etcd**: the key-value store, used for backs-up and keeping manifests for the entire cluster

There are two additional components on the control plane, they are **kubelet** and **k-proxy**. These two are special and important as they are installed on all node. You can see the Data Plane below for more details.

### Data plane

<img src="imgs/le03ex09im03.png" title="Data Plane conponents"  />

The data plane consists of the compute used to host workloads. The components installed on a worker node are the:
- **kubelet**: the agent that runs on every node and notifies the kube- apiserver that this node is part of the cluster
- **kube-proxy**: a network proxy that ensures the reachability and accessibility of workloads places on this specific node

**Important Note**: The kubelet and kube-proxy components are installed on all the nodes in the cluster (master and worker nodes). These components keep the kube-apiserver up-to-date with a list of nodes in the cluster and manages the connectivity and reachability of the workloads.

## Concept 11: Deploy Your First Kubernetes Cluster

### New terms
**Bootstrap** - the process of provisioning a Kubernetes cluster, by ensuring that each node has the necessary components to be fully operational

### Further reading
[Bootstrapping clusters with kubeadm](https://kubernetes.io/docs/setup/production-environment/tools/kubeadm/) - a step-by-step guide on how to use kubeadm to provision a cluster

---

To handle the bootstrapping of a cluster automatically:
- **Production**-grade clusters
    - [kubeadm](https://kubernetes.io/docs/setup/production-environment/tools/kubeadm/install-kubeadm/)
    - [Kubespray](https://github.com/kubernetes-sigs/kubespray)
    - [Kops](https://github.com/kubernetes/kops)
    - [K3s](https://k3s.io/)
- **Developmnet**-grade clusters
    - [kind](https://kind.sigs.k8s.io/docs/user/quick-start/)
    - [minikube](https://minikube.sigs.k8s.io/docs/start/)
    - [k3d](https://k3d.io/)
    
A good introduction to k3d, written by k3d's creator Thorsten Klein, can be found [here](https://www.suse.com/c/introduction-k3d-run-k3s-docker-src/). 

A comprehensive overview of currently existing lightweight kubernetes distros can be found [here](https://www.suse.com/c/running-a-local-kubernetes-cluster-k3s-src).



### Create Vagrant Box And Install Kubernetes with k3s

This demo is a step-by-step guide on how to create a vagrant box and install a Kubernetes cluster using k3s. 

To follow this demo, reference the [Vagrantfile](https://github.com/udacity/nd064_course_1/blob/main/exercises/Vagrantfile) from the course repository.

A nice introduction to Vagrant can be found in [this article](https://community.suse.com/posts/vagrant-never-gets-old).

Prepare:
- [VirtualBox](https://www.virtualbox.org/wiki/Downloads) > `6.1.16`
- install [Vagrant](https://www.vagrantup.com/)

**Note**: I had a conflict with a Mac Aiport ip and a Vagrant network ip. I changed in a Vagrant file ip from `192.168.50.4` to `192.168.51.4`. Simple remove the virtual image if someone goes wrong.

```
Bridged Network Address: '192.168.50.0'
Host-only Network 'en0: Wi-Fi (AirPort)': '192.168.50.0'
```

```bash
cd <to nd064_cource_1>/exercises

# version
vagrant version

# Running the vmachine
vagrant up

# Status the vmachine
vagrant status

# SSH into the vagrant box
# Note: this command uses the .vagrant folder to identify the details of the vagrant box
vagrant ssh
```

After ssh into vagrant box follow to [k3s.io](https://k3s.io/) and install cluster in the vbox cli:
```
vagrant@localhost:~> curl -sfL https://get.k3s.io | sh

sudo su

# Check claster
kubectl get no
```


## Concept 12: Kubeconfig

### New terms
**Kubeconfig** - a metadata file that grants a user access to a Kubernetes cluster

### Further reading
[Organizing Cluster Access Using kubeconfig Files](https://kubernetes.io/docs/concepts/configuration/organize-cluster-access-kubeconfig/)

---

A kubeconfig file has all the necessary cluster metadata and authentication details, that grants the user permission to query the cluster objects.

- `~/.kube/config`
    - k3s: `/etc/rancher/k3s/k3s.yaml`
- `--kubeconfig` flag
- `KUBECONFIG` environment variable

A Kubeconfig file has 3 main distinct sections:
- **Cluster** - encapsulates the metadata for a cluster, such as the name of the cluster, API server endpoint, and certificate authority used to check the identity of the user.
- **User** - contains the user details that want access to the cluster, including the user name, and any authentication metadata, such as username, password, token or client, and key certificates.
- **Context** - links a user to a cluster. If the user credentials are valid and the cluster is up, access to resources is granted. Also, a `current-context` can be specified, which instructs which context (cluster and user) should be used to query the cluster.


Kubeconfig example:
```
apiVersion: v1
# define the cluster metadata 
clusters:
- cluster:
    certificate-authority-data: {{ CA }}
    server: https://127.0.0.1:63668
  name: udacity-cluster
# define the user details 
users:
# `udacity-user` user authenticates using client and key certificates 
- name: udacity-user
  user:
    client-certificate-data: {{ CERT }}
    client-key-data: {{ KEY }}
# `green-user` user authenticates using a token
- name: green-user
  user:
    token: {{ TOKEN }}
# define the contexts 
contexts:
- context:
    cluster: udacity-cluster
    user: udacity-user
  name: udacity-context
# set the current context
current-context: udacity-context
```

Once you start handling multiple clusters, you'll find a lot of useful information in [this article](https://community.suse.com/posts/scheduled/cluster-this-is-your-admin-do-you-read)



## Exercise 14: Deploy Your First Kubernetes Cluster

The **kubeconfig file** and `kubectl` commands are the 2 main components that permits the interaction with a Kubernetes cluster.

**kubeconfig**:
- K3s stores the kubeconfig file under `/etc/rancher/k3s/k3s.yaml` path
- API server: `https://127.0.0.1:6443`
- authentication mechanism: username (admin) and password

**kubectl commands**:
- `kubectl cluster-info`: to get the control plane and add-ons endpoints
```
Kubernetes master is running at https://127.0.0.1:6443
CoreDNS is running at https://127.0.0.1:6443/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy
Metrics-server is running at https://127.0.0.1:6443/api/v1/namespaces/kube-system/services/https:metrics-server:/proxy
```
- `kubectl get nodes`: to get all the nodes in the cluster
```
NAME        STATUS   ROLES    AGE   VERSION
localhost   Ready    master   74m   v1.18.9+k3s1
```
- `kubectl get nodes -o wide`: to get extra details about the nodes, including internal IP
```
NAME        STATUS   ROLES    AGE   VERSION        INTERNAL-IP   EXTERNAL-IP   OS-IMAGE             KERNEL-VERSION            CONTAINER-RUNTIME
localhost   Ready    master   74m   v1.18.9+k3s1   10.0.2.15     <none>        openSUSE Leap 15.2   5.3.18-lp152.47-default   containerd://1.3.3-k3s2
```
- `kubectl describe node node-name`: to get all the configuration details about the node, including the allocated pod CIDR
```
kubectl describe node localhost | grep CIDR
PodCIDR:                      10.42.0.0/24
PodCIDRs:                     10.42.0.0/24
```

## Concept 16: Kubernetes Resources Part 1

### New terms
- **Pod** - smallest manageable uint within a cluster that provides the execution environment for an application
- **ReplicaSet** - a mechanism to ensure a number of pod replicas are up and running at all times
- **Deployment** - describe the desired state of the application to be deployed

### Further reading
Explore the Kubernetes resources in more detail:
- [Kubernetes Pods](https://kubernetes.io/docs/concepts/workloads/pods/)
- [Kubernetes Deployments](https://kubernetes.io/docs/concepts/workloads/controllers/deployment/)
- [Kubernetes ReplicaSets](https://kubernetes.io/docs/concepts/workloads/controllers/replicaset/)
- [Kubernetes RollingOut Strategies](https://kubernetes.io/docs/concepts/workloads/controllers/deployment/#strategy)
---

Kubernetes provides a rich collection of resources that are used to deploy, configure, and manage an application. Some of the widely used resources are:
- **Pods**: the atomic element within a cluster to manage an application
- **Deployments & ReplicaSets**: oversees a set of pods for the same application
- **Services & Ingress** ensures connectivity and reachability to pods
- **Configmaps & Secrets** pass configuration to pods
- **Namespaces**: provides a logical separation between multiple applications and their resources
- **Custom Resource Definition (CRD)**: - extends Kubernetes API to support custom resources

### Application Deployment

<img src="imgs/le03ex16im01.png" title="Pod architecture, showcasing a container running a Docker image"  />

A **pod** is the anatomic element within a cluster that provides the execution environment for an application. Pods are the smallest manageable units in a Kubernetes cluster. Every pod has a container within it, that executes an application from a Docker image (or any OCI-compliant image). There are use cases where 2-3 containers run within the same pod, however, it is highly recommended to keep the 1:1 ratio between your pods and containers.

All the pods are placed on the cluster nodes. A note can host multiple pods for different applications.


### Deployments and ReplicaSets

<img src="imgs/le03ex16im02.png" title="Application management using a Deployment and ReplicaSet"  />

To deploy an application to a Kubernetes cluster, a **Deployment** resource is necessary. A Deployment contains the specifications that describe the desired state of the application. Also, the Deployment resource manages pods by using a **ReplicaSet**. A ReplicaSet resource ensures that the desired amount of replicas for an application are up and running at all times.

To create a deployment, use the `kubectl create deployment` command, with the following syntax:
```bash
# create a Deployment resource
# NAME - required; set the name of the deployment
# IMAGE - required;  specify the Docker image to be executed
# FLAGS - optional; provide extra configuration parameters for the resource
# COMMAND and args - optional; instruct the container to run specific commands when it starts 
kubectl create deploy NAME --image=image [FLAGS] -- [COMMAND] [args]

# Some of the widely used FLAGS are:
-r, --replicas - set the number of replicas
-n, --namespace - set the namespace to run
--port - expose the container port

# create a go-helloworld Deployment in namespace `test`
kubectl create deploy go-helloworld --image=pixelpotato/go-helloworld:v1.0.0 -n test
```

To create a headless pod, the `kubectl run` command is handy, with the following syntax:
```bash
# create a headless pod
# NAME - required; set the name of the pod
# IMAGE - required;  specify the Docker image to be executed
# FLAGS - optional; provide extra configuration parameters for the resource
# COMMAND and args - optional; instruct the container to run specific commands when it starts 
kubectl run NAME --image=image [FLAGS] -- [COMMAND] [args...]

# Some of the widely used FLAGS are:
--restart - set the restart policy. Options [Always, OnFailure, Never]
--dry-run - dry run the command. Options [none, client, server]
-it - open an interactive shell to the container

# example: create a busybox pod, with an interactive shell and a restart policy set to Never 
kubectl run -it busybox-test --image=busybox --restart=Never
```

### Rolling Out Strategy

<img src="imgs/le03ex16im03.png" title="Rolling update of an application, between different versions"  />

The Deployment resource comes with a very powerful rolling out strategy, which ensures that no downtime is encountered when a new version of the application is released. Currently, there are 2 rolling out strategies:
- **RollingUpdate**: updates the pods in a rolling out fashion (e.g. 1-by-1)
- **Recreate**: kills all existing pods before new ones are created

## Kubernetes Resources Part 2


### New terms
- **Service** - an abstraction layer over a collection of pods running an application
- **Ingress** - a mechanism to manage the access from external users and workloads to the services within the cluster

### Further reading
Explore Kubernetes resources used to connect to an application:
- [Kubernetes Services](https://kubernetes.io/docs/concepts/services-networking/service/)
- [Kubernetes Ingress](https://kubernetes.io/docs/concepts/services-networking/ingress/)
---

### Application Reachability

Within a cluster, every pod is allocated 1 unique IP which ensures connectivity and reachability to the application inside the pod. This IP is only routable inside the cluster, meaning that external users and services will not be able to connect to the application.

### Services

<img src="imgs/le03ex17im01.png" title="Pods accessibility through a Service resource"  />

A **Service** resource provides an abstraction layer over a collection of pods running an application. A Service is allocated a cluster IP, that can be used to transfer the traffic to any available pods for an application.

As such, as shown in the above image, instead of accessing each pod independently, the workload (1) should access the service IP (2), which routes the requests to available pods (3).

There are 3 widely used Service types:
- **ClusterIP**: exposes the service using an internal cluster IP. If no service type is specified, a ClusterIP service is created by default.
- **NodePort**: expose the service using a port exposed on all nodes in the cluster.
- **LoadBalancer**: exposes the service through a load balancer from a public cloud provider such as AWS, Azure, or GCP. This will allow the external traffic to reach the services within the cluster securely.

To create a service for an existing deployment, use the `kubectl expose deployment` command, with the following syntax:
```bash
# expose a Deployment through a Service resource 
# NAME - required; set the name of the deployment to be exposed
# --port - required; specify the port that the service should serve on
# --target-port - optional; specify the port on the container that the service should direct traffic to
# FLAGS - optional; provide extra configuration parameters for the service
kubectl expose deploy NAME --port=port [--target-port=port] [FLAGS]

# Some of the widely used FLAGS are:
--protocol - set the network protocol. Options [TCP|UDP|SCTP]
--type - set the type of service. Options [ClusterIP, NodePort, LoadBalancer]

# expose the `go-helloworld` deployment on port 8111
# note: the application is serving requests on port 6112
kubectl expose deploy go-helloworld --port=8111 --target-port=6112
```


### Ingress

<img src="imgs/le03ex17im02.png" title="Ingress resources enabling access from the external users to services within the cluster"  />

To enable the external user to access services within the cluster an **Ingress** resource is necessary. An Ingress exposes HTTP and HTTPS routes to services within the cluster, using a load balancer provisioned by a cloud provider. Additionally, an Ingress resource has a set of rules that are used to map HTTP(S) endpoints to services running in the cluster. To keep the Ingress rules and load balancer up-to-date an Ingress Controller is introduced.

For example, as shown in the image above, the customers will access the go-helloworld.com/hi HTTP route (1), which is managed by an Ingress (2). The Ingress Controller (3) examines the configured routes and directs the traffic to a LoadBalancer (4). And finally, the LoadBalancer directs the requests to the pods using a dedicated port (5).

## Kubernetes Resources Part 3

### New terms
- **Configmap** - a resource to store non-confidential data in key-value pairs.
- **Secret** - a resource to store confidential data in key-value pairs. These are base64 encoded.
- **Namespace** - a logical separation between multiple applications and associated resources.

### Further reading
Explore Kubernetes resources to pass configuration to pods:
- [Kubernetes Configmap](https://kubernetes.io/docs/concepts/configuration/configmap/)
- [Kubernetes Secrets](https://kubernetes.io/docs/concepts/configuration/secret/)
- [Kuebrnetes Namespaces](https://kubernetes.io/docs/concepts/overview/working-with-objects/namespaces/)

### ConfigMaps

**ConfigMaps** are objects that store non-confidential data in key-value pairs. A Configmap can be consumed by a pod as an environmental variable, configuration files through a volume mount, or as command-line arguments to the container.

To create a Configmap use the `kubectl create configmap` command, with the following syntax:
```bash
# create a Configmap
# NAME - required; set the name of the configmap resource
# FLAGS - optional; define  extra configuration parameters for the configmap
kubectl create configmap NAME [FLAGS]

# Some of the widely used FLAGS are:
--from-file - set path to file with key-value pairs 
--from-literal - set key-value pair from command-line 

# create a Configmap to store the color value
kubectl create configmap test-cm --from-literal=color=yellow
```

### Secrets

**Secrets** are used to store and distribute sensitive data to the pods, such as passwords or tokens. Pods can consume secrets as environment variables or as files from the volume mounts to the pod. It is noteworthy, that Kubernetes will encode the secret values using base64.

To create a Secret use the `kubectl create secret generic` command, with the following syntax:
```bash
# create a Secret
# NAME - required; set the name of the secret resource
# FLAGS - optional; define  extra configuration parameters for the secret
kubectl create secret generic NAME [FLAGS]

# Some of the widely used FLAGS are:
--from-file - set path to file with the sensitive key-value pairs 
--from-literal - set key-value pair from command-line 

# create a Secret to store the secret color value
kubectl create secret generic test-secret --from-literal=color=blue
```

### Namespaces

A Kubernetes cluster is used to host hundreds of applications, and it is required to have separate execution environments across teams and business verticals. This functionality is provisioned by the Namespace resources. A **Namespace** provides a logical separation between multiple applications and associated resources. In a nutshell, it provides the **application context**, defining the environment for a group of Kubernetes resources that relate to a project, such as the amount of CPU, memory, and access. For example, a project-green namespace includes any resources used to deploy the Green Project. These resources construct the application context and can be managed collectively to ensure a successful deployment of the project.

Each team or business vertical is allocated a separate Namespace, with the desired amount of CPU, memory, and access. This ensures that the application is managed by the owner team and has enough resources to execute successfully. This also eliminates the "noisy neighbor" use case, where a team can consume all the available resources in the cluster if no Namespace boundaries are set.

To create a Namespace we can use the `kubectl create namespace` command, with the following syntax:
```bash
# create a Namespace
# NAME - required; set the name of the Namespace
kubectl create ns NAME

# create a `test-udacity` Namespace
kubectl create ns test-udacity

# get all the pods in the `test-udacity` Namespace
kubectl get po -n test-udacity
```

## Concept 19: Useful kubectl commands

- **RESOURCE** is the Kubernetes resource type
- **NAME** sets the name of the resource
- **FLAGS** are used to provide extra configuration
- **PARAMS** are used to provide the required configuration to the resource

```bash
# Create Resources
kubectl create RESOURCE NAME [FLAGS]

# Describe Resources
kubectl describe RESOURCE NAME 

# Get Resources
kubectl get RESOURCE NAME [-o yaml]

# Edit Resources
kubectl edit RESOURCE NAME [-o yaml]

# Label Resources
kubectl label RESOURCE NAME [PARAMS]

# Port-forward to Resources
kubectl port-forward RESOURCE/NAME [PARAMS]

#Logs from Resources
kubectl logs RESOURCE/NAME [FLAGS]

# Delete Resources
kubectl delete RESOURCE NAME
```

## Exercice 20: k8s resources

- Application management: Deployments, ReplicaSets and Pods
- Application reachability: Services and Ingress
- Application configuration: Secrets and ConfigMap
- Application context: Namespaces




## Exercise 21: Kubernetes Resources

Now you have learned many Kubernetes recourses, in this exercise, you will deploy the following resources using the `kubectl` command.

a namespace
- name: `demo`
- label: `tier: test`

a deployment:
- image: `nginx:alpine`
- name: `nginx-apline`
- namespace: `demo`
- replicas: 3
- labels: `app: nginx, tag: alpine`

a service:
- expose the above deployment on port 8111
- namespace: `demo`

a configmap:
- name: `nginx-version`
- containing key-value pair: `version=alpine`
- namespace: `demo`

Note: Nginx is one of the public Docker images, that you can access and use for your exercises or testing purposes.

```bash
# create the namespace 
# note: label option is not available with `kubectl create`
kubectl create ns demo

# label the namespace
kubectl label ns demo tier=test

# create the nginx-alpine deployment 
kubectl create deploy nginx-alpine --image=nginx:alpine  --replicas=3 --namespace demo

# label the deployment
kubectl label deploy nginx-alpine app=nginx tag=alpine --namespace demo

# expose the nginx-alpine deployment, which will create a service
kubectl expose deployment nginx-alpine --port=8111 --namespace demo

# create a config map
kubectl create configmap nginx-version --from-literal=version=alpine --namespace demo
```

## Concept 23: Declarative Kubernetes Manifests

### New terms
- **Imperative configuration** - resource management technique, that operates and interacts directly with the live objects within the cluster.
- **Declarative configuration** - resource management technique, that operates and manages resources using YAML manifests stored locally.

### Further reading
Explore more details about different management techniques and advanced configuration for Deployment resources:
- [Managing Kubernetes Objects Using Imperative Commands](https://kubernetes.io/docs/tasks/manage-kubernetes-objects/imperative-command/)
- [Declarative Management of Kubernetes Objects Using Configuration Files](https://kubernetes.io/docs/tasks/manage-kubernetes-objects/declarative-config/)
- [Configure Liveness, Readiness Probes for a Deployment](https://kubernetes.io/docs/tasks/configure-pod-container/configure-liveness-readiness-startup-probes/)
- [Managing Resources for Containers](https://kubernetes.io/docs/concepts/configuration/manage-resources-containers/)
---


A YAML manifest consists of 4 obligatory sections:
- **apiversion** - API version used to create a Kubernetes object
- **kind** - object type to be created or configured
- **metadata** - stores data that makes the object identifiable, such as its name, namespace, and labels
- **spec** - defines the desired configuration state of the resource

```YAML
## Set the API endpoint used to create the Deployment resource.
apiVersion: apps/v1
## Define the type of the resource.
kind: Deployment
## Set the parameters that make the object identifiable, such as its name, namespace, and labels.
metadata:
  annotations:
  labels:
    app: go-helloworld
  name: go-helloworld
  namespace: default
## Define the desired configuration for the Deployment resource.
spec:
  ## Set the number of replicas.
  ## This will create a ReplicaSet that will manage 3 pods of the Go hello-world application.
  replicas: 3
  ## Identify the pods managed by this Deployment using the following selectors.
  ## In this case, all pods with the label `go-helloworld`.
  selector:
    matchLabels:
      app: go-helloworld
  ## Set the RollingOut strategy for the Deployment.
  ## For example, roll out only 25% of the new pods at a time.
  strategy:
    rollingUpdate:
      maxSurge: 25%
      maxUnavailable: 25%
    type: RollingUpdate
  ## Set the configuration for the pods.
  template:
    ## Define the identifiable metadata for the pods.
    ## For example, all pods should have the label `go-helloworld`
    metadata:
      labels:
        app: go-helloworld
    ## Define the desired state of the pod configuration.
    spec:
      containers:
        ## Set the image to be executed inside the container and image pull policy
        ## In this case, run the `go-helloworld` application in version v2.0.0 and
        ## only pull the image if it's not available on the current host.
      - image: pixelpotato/go-helloworld:v2.0.0
        imagePullPolicy: IfNotPresent
        name: go-helloworld
        ## Expose the port the container is listening on.
        ## For example, exposing the application port 6112 via TCP.
        ports:
        - containerPort: 6112
          protocol: TCP
        ## Define the rules for the liveness probes.
        ## For example, verify the application on the main route `/`,
        ## on application port 6112. If the application is not responsive, then the pod will be restarted automatically. 
        livenessProbe:
           httpGet:
             path: /
             port: 6112
        ## Define the rules for the readiness probes.
        ## For example, verify the application on the main route `/`,
        ## on application port 6112. If the application is responsive, then traffic will be sent to this pod.
        readinessProbe:
           httpGet:
             path: /
             port: 6112
        ## Set the resource requests and limits for an application.
        resources:
        ## The resource requests guarantees that the desired amount 
        ## CPU and memory is allocated for a pod. In this example, 
        ## the pod will be allocated with 64 Mebibytes and 250 miliCPUs.
          requests:
            memory: "64Mi"
            cpu: "250m"
        ## The resource limits ensure that the application is not consuming 
        ## more than the specified CPU and memory values. In this example, 
        ## the pod will not surpass 128 Mebibytes and 500 miliCPUs.
          limits:
            memory: "128Mi"
            cpu: "500m"
```


[K8S Cheatsheet](https://kubernetes.io/docs/reference/kubectl/cheatsheet/)